In [20]:
# import libraries

import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)

%run ./scripts/util_func.py
%run ./scripts/model_func.py

In [21]:
# import arrays

icp_states = np.load('data/icp_states.npy') # [time, icp_index, icp_x, icp_y, icp_yaw]
wheel_vels = np.load('data/wheel_vels.npy') # [time, icp_index, vel_l, vel_r]

print(icp_states)

[[ 0.00000000e+00  2.00000000e+00  1.90564990e-03  6.57618046e-04
   7.89723646e-01]
 [ 4.63672320e-02  3.00000000e+00  4.82946634e-04 -3.39448452e-05
   7.89840095e-01]
 [ 9.66886400e-02  4.00000000e+00  4.24474478e-04 -4.95791435e-04
   7.89964715e-01]
 ...
 [ 8.71247166e+02  8.71800000e+03  3.89616919e+00  5.62076807e+00
  -2.41419225e+00]
 [ 8.71299086e+02  8.71900000e+03  3.89392924e+00  5.62077951e+00
  -2.41430764e+00]
 [ 8.71349671e+02  8.71900000e+03  3.89392924e+00  5.62077951e+00
  -2.41430764e+00]]


In [22]:
# Optimize icr params for entire trajectory

# params from crv
r = 0.3 # wheel radius : meters
alpha_l = 0.81 # left side friction loss
alpha_r = 0.84 # right side friction loss
x_icr = -2.71 # vehicle x_icr
y_icr_l = 3.0 # left wheels icr in y coordinates
y_icr_r = -3.85 # right wheels icr in y coordinates
k = np.array([alpha_l, alpha_r, x_icr, y_icr_l, y_icr_r])
# k = np.array([1.0, 1.0,  1.0, 1.0, -1.0])

def compute_err_traj(k):
    err = 0
    edd_pose = icp_states[0, 2:]
    vel_count = 0
    icp_disp = 0
    
#     propa_mat = np.array([[np.sin(icp_states[0,4]), 0.0, 0.0], 
#                       [0.0, np.cos(icp_states[0,4]), 0.0], [0.0, 0.0, 1.0]])
    propa_cos = np.cos(icp_states[0,4])
    propa_sin = np.sin(icp_states[0,4])
    propa_mat = np.array([[propa_cos, -propa_sin, 0.0], 
                      [propa_sin, propa_cos, 0.0], [0.0, 0.0, 1.0]])
    
    for i in range(icp_states.shape[0] - 1):
        
        dt = icp_states[i+1, 0] - icp_states[i, 0]
        vel_count = vel_count + 1
        icp_disp = icp_disp + comp_disp(icp_states[i+1, 2:3], icp_states[i, 2:3])
        
        propa_mat = up_propa_mat(propa_mat, edd_pose[2])
        
        edd_pose = edd_pose + propa_mat @ icr_diff_drive(wheel_vels[i, 2:], k) * dt
        edd_pose[2] = wrap2pi(edd_pose[2])
        
        if icp_disp >= 2.0:
            err = err + disp_err(edd_pose, icp_states[i+1, 2:])
            icp_disp = 0
            edd_pose = icp_states[i+1, 2:]
            vel_count = 0
    return err

print('initial error : ', compute_err_traj(k))

res = minimize(compute_err_traj, k, method='nelder-mead')
print(res)



initial error :  3733.844355504954
 final_simplex: (array([[ 0.62338352,  0.65553787, -0.85653332,  2.75210003, -2.7313849 ],
       [ 0.62337872,  0.65553285, -0.85658455,  2.75214948, -2.73129336],
       [ 0.62336696,  0.65552048, -0.85653882,  2.75202438, -2.73131496],
       [ 0.62337155,  0.65552534, -0.85655504,  2.75207356, -2.73130635],
       [ 0.62337277,  0.65552662, -0.85654892,  2.75207324, -2.73131739],
       [ 0.62337308,  0.65552696, -0.85655252,  2.75208071, -2.73131271]]), array([2814.63058429, 2814.63058562, 2814.63058721, 2814.63058795,
       2814.63058841, 2814.63058876]))
           fun: 2814.630584289276
       message: 'Optimization terminated successfully.'
          nfev: 731
           nit: 452
        status: 0
       success: True
             x: array([ 0.62338352,  0.65553787, -0.85653332,  2.75210003, -2.7313849 ])


In [23]:
# Optimize icr params for entire trajectory

# params from crv
r = 0.3 # wheel radius : meters
alpha_l = 0.81 # left side friction loss
alpha_r = 0.84 # right side friction loss
x_icr = -2.71 # vehicle x_icr
y_icr_l = 3.0 # left wheels icr in y coordinates
y_icr_r = -3.85 # right wheels icr in y coordinates
k = np.array([alpha_l, alpha_r, x_icr, y_icr_l, y_icr_r])
# k = np.array([-1.96629809e-08, -1.47389374e-08,  9.16716929e-01,  5.19340378e+00,
#        -1.07375266e+01])

def compute_err_seg(k):
    index = index_start
    while index <= index_end:
        dt = icp_states[i+1, 0] - icp_states[index, 0]
        propa_mat = up_propa_mat(propa_mat, edd_pose[2])
        edd_pose = edd_pose + propa_mat @ icr_diff_drive(wheel_vels[index, 2:], k) * dt
        
        index = index + 1
    
    err = disp_err(edd_pose, icp_states[index+1, 2:])
    return err

err = 0
edd_pose = icp_states[0, 2:]
vel_count = 0
icp_disp = 0
index_start = 0

k_arr = []

propa_mat = np.array([[np.sin(icp_states[0,4]), 0.0, 0.0], 
                      [0.0, np.cos(icp_states[0,4]), 0.0], [0.0, 0.0, 1.0]])
    
for i in range(icp_states.shape[0] - 1):
    
    icp_disp += comp_disp(icp_states[i+1, 2:3], icp_states[i, 2:3])
    
    if icp_disp >= 2.0:
        index_end = i
        res_sec = minimize(compute_err_seg, k, method='nelder-mead')
        k_arr.append(res_sec.x)
        index_start = i
        icp_disp = 0
    

UnboundLocalError: local variable 'propa_mat' referenced before assignment